In [1]:
#import statements
import numpy as np
import math
import re

#PARAMETER #1; filenames
#file name you'll save to
filename = 'SineBenchy.gcode'
#file name you'll read
readname = 'benchy.gcode'

#PARAMETER #2; sliced Gcode settings
#These are the params the input GCODE was sliced with
layerHeight = 0.3
width = 0.6

with open(filename, 'w') as file:
    file.write('; This code was generated by Leviticus Rhodens Non-planar code.\n ; Website: https://leviticusrhoden.com/crafting/nonPlanar3D.php \n ; Email me: levi@leviticusrhoden.com \n')


These functions write to the output GCODE file in different ways.

In [2]:
#This cell is a collection of add to GCODE commands
def addPoint(x,y,z,e,f):
    '''This appends a new co-ordinate to the gcode file! It takes in x,y,z,and e co-ordinates as floats.'''
    
    with open(filename,'a') as file:
        if z<0:
            print("ERROR: I'm under the print bed!")
            return
        if e==0:
            file.write('G0 ')
        else:
            file.write('G1 ')
        file.write('F' + "{:#.8g}".format(f) + ' X' + "{:#.8g}".format(x) + ' Y' + "{:#.8g}".format(y) + ' Z' + "{:#.8g}".format(z) + ' E' + "{:#.8g}".format(e) + '\n')
        
def addComment(note):
    '''Takes in a string note and adds it as a comment to the GCODE file, does not need to have the ; in front'''
    with open(filename,'a') as file:
        file.write(";" + note +"\n")

def addString(string):
    '''Takes in a string and adds it to the output file, no questions asked and no notation added'''
    with open(filename,'a') as file:
        file.write(string)

These functions define the surfaces we want to interpolate between

In [3]:
#these functions define the surface maps we are transitioning between
#just make sure that the map at each point (0, z2 and z3) dosn't overlap with the other maps.
#ALso that Zhop in origonal code is higher than any non-linear function here

#PARAMETER #3, heights of different layer surfaces
z2 = 48
z3 = 200

#PARAMETER #4; The functions that define the layer surfaces. Note, they don't need to be smooth! Be careful but have fun!
def surface2(x,y):
    return 5*np.sin((x-175)/2.5)*np.cos((y-175)/5)
    
def surface3(x,y):
    return surface2(x,y)


def surface(x,y,z):
    '''This takes in the x,y, and planar z float, and returns the deltaZ from the planar z'''
    if z<z2:
        dz = (z/z2)*surface2(x,y)
    else:
        dz = (1-(z-z2)/(z3-z2))*surface2(x,y)
    return dz

This function calculates the control volume of a pair of G1 commands

In [4]:
def calcControlVolume(X, Y, lastX, lastY, height, E):
    '''This Function takes in current and last co-ords, and calculates new, non-planar co-ordinates and extrsion ratio.'''
    Z = height
    
    #This calculates volume of extrusion control volume and the planar volume, then finds the ratio of the two
    #This ratio is multiplied by the old extrusion value, preserving over/under extrusion in og GCODE
    #(GCODE takes a linear extrusion)
    dX = ((X-lastX)**2 + (Y-lastY)**2)**0.5
    dZ = surface(X,Y,Z) - surface(X,Y,Z-layerHeight) +layerHeight
    lastDZ = surface(lastX,lastY,Z) - surface(lastX,lastY,Z-layerHeight) + layerHeight
    vol = dX*width*0.5*(dZ + lastDZ)
    oldVol = dX*width*layerHeight
    
    if oldVol > 0:
        rVol = vol/oldVol
    else:
        rVol = 1
    
    return X, Y, Z+surface(X,Y,Z), rVol
            

These functions disect GCODE for floats

In [5]:
def extractCords(string):
    '''This function takes in a line of GCODE as a string and outputs co-ordinates in that string'''
    
    out = ['nan','nan','nan','nan','nan']
    coord = re.findall(r'[XYZEF].?\d+.\d+', string)
    
    for i in coord:
        if 'X' in i:
            out[0] = float(i[1:])
        if 'Y' in i:
            out[1] = float(i[1:])
        if 'Z' in i:
            out[2] = float(i[1:])
        if 'E' in i:
            out[3] = float(i[1:])
        if 'F' in i:
            out[4] = float(i[1:])
            
    return out
            
    

This guy puts it all together!

In [6]:
#This loop iterates over every line of text in the input GCODE, and modifies the code so movement is non planar.
#it also corrects extrusion rate

layer = 1
height = layerHeight
lastCoords = [150,150,100,0,3000]
totalE = 0

with open(readname) as file:
    for line in file:
        #This level is called for every line of input GCODE
        # This if checks for a move command   
        if 'G1 ' in line or 'G0 ' in line:
            #get co-ordinates from said line
            coords = extractCords(line)
            #look through co-ords and set to last co-ord if it is not mentioned in the line. This is how GCODE is writen for simplicity
            if coords[0] == 'nan':
                coords[0] = lastCoords[0]
                addComment("X was the same...")
            if coords[1] == 'nan':
                coords[1] = lastCoords[1]
            if coords[2] == 'nan':
                coords[2] = lastCoords[2]
            if coords[3] == 'nan':
                coords[3] = lastCoords[3]
            
            dist = ((coords[0]-lastCoords[0])**2 + (coords[1]-lastCoords[1])**2)**0.5
            maxLength = .5
            if dist >= maxLength:
                iterations = int(dist/maxLength)
            else:
                iterations = 1

            lastX = lastCoords[0]
            lastY = lastCoords[1]

            #The following comment is saved for posterity. Please enjoy the rage comment:
            #TODO PISS HERE THING MAYBE??? change 1 to 0????

            for i in list(range(1, iterations+1, 1)):
                #print(i, iterations)
                t = i/iterations
                x = t*coords[0] + (1-t)*lastCoords[0]
                y = t*coords[1] + (1-t)*lastCoords[1]
                
                xPt, yPt, zPt, rVol = calcControlVolume(x, y, lastX, lastY, height, 0)
                e = rVol*(coords[3])/(iterations)
                
                if True:
                    totalE = e + totalE
                
                addPoint(xPt, yPt, zPt, e, 3000)
                
                lastX = x
                lastY = y
            lastCoords[0] = lastX
            lastCoords[1] = lastY
            for i in [2,3,4]:
                    if True:
                        lastCoords[i] = coords[i]
            
        elif ';LAYER' in line:
            print(layer)
            layer += 1
            height = (layer-1)*layerHeight
            addString(line)
        else:
            addString(line)
            


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
